In [1]:
from model import *
import torch
import torch.nn as nn
from torch.autograd import Variable

In [2]:
randomStuff = torch.rand(5, 3, 16, 16)

# conv test

In [21]:
kernel = 2
stride = 3
padding = 1
in_channels = 3
out_channels =48
randomStuff = torch.rand(10, in_channels, 25, 25)
ourConv = Conv2d(in_channels, out_channels, kernel, stride, padding)
torchConv = nn.Conv2d(in_channels, out_channels, kernel, stride, padding)

In [22]:

ourConv.weights = torchConv.weight
ourConv.bias = torchConv.bias

In [23]:
ourOutput = ourConv.forward(randomStuff)
torchOutput = torchConv(randomStuff)
torch.testing.assert_allclose(torchOutput, ourOutput)

In [24]:
x=Variable(randomStuff,requires_grad=True)
x2=Variable(x*1.0,requires_grad=True)

In [25]:
x.grad

In [26]:
o=ourConv.forward(x2)

In [27]:
p=torchConv(x)

In [28]:
grdw=torch.rand(p.shape)

In [29]:


p.backward(grdw)

In [30]:
a=ourConv.backward(grdw)

In [31]:
torch.testing.assert_allclose(a, x.grad)

In [32]:
torch.testing.assert_allclose(torchConv.weight.grad,ourConv.dweights)


In [33]:
torch.testing.assert_allclose(torchConv.bias.grad,ourConv.dbias)

In [35]:
torchConv.bias.grad

tensor([417.8065, 402.0136, 402.9307, 396.2509, 428.6507, 409.7196, 394.5825,
        410.5635, 412.7366, 407.8360, 405.2646, 407.4084, 401.7672, 402.5669,
        400.5905, 397.1891, 407.7005, 387.9539, 391.0354, 401.2856, 389.5426,
        405.0171, 421.5373, 407.3176, 401.0757, 413.8104, 407.7632, 400.5075,
        385.0281, 406.4738, 399.6791, 411.9471, 401.3462, 410.1255, 399.9250,
        402.8688, 394.9614, 404.8026, 407.2644, 416.3716, 401.6061, 397.4891,
        400.5943, 403.0455, 406.2382, 405.8355, 393.1065, 405.1621])

In [34]:
ourConv.dbias

tensor([417.8066, 402.0137, 402.9308, 396.2509, 428.6508, 409.7197, 394.5824,
        410.5634, 412.7368, 407.8359, 405.2647, 407.4084, 401.7672, 402.5669,
        400.5906, 397.1893, 407.7006, 387.9539, 391.0354, 401.2855, 389.5424,
        405.0171, 421.5372, 407.3176, 401.0758, 413.8102, 407.7631, 400.5074,
        385.0281, 406.4738, 399.6792, 411.9472, 401.3464, 410.1255, 399.9251,
        402.8688, 394.9614, 404.8026, 407.2645, 416.3716, 401.6061, 397.4892,
        400.5943, 403.0456, 406.2380, 405.8353, 393.1066, 405.1621])

# upsample test

In [3]:
scale_factor = 2
loss=nn.MSELoss()
ourUpsample = NearestUpsampling(scale_factor=scale_factor)
torchUpsample = nn.Upsample(scale_factor = scale_factor)

In [8]:
ourOutput = ourUpsample.forward(randomStuff)
torchOutput = torchUpsample(randomStuff)
torch.testing.assert_allclose(torchOutput, ourOutput)

In [5]:
x=Variable(torch.rand(5*3*16).view((5,3,4,4))*1.0,requires_grad=True)

In [6]:
outp=torch.rand(5*3*8*8).view(5,3,8,8)

In [7]:
grdout=torch.rand_like(torchUpsample(x))

In [23]:
from torch.autograd import Variable

In [9]:
torchUpsample(x).backward(grdout)

In [12]:
torch.testing.assert_allclose(ourUpsample.backward(grdout),x.grad)

# MSE test

In [60]:
ourMSE = MSE()
torchMSE = nn.MSELoss()

In [61]:
x=Variable(randomStuff,requires_grad=True)

In [62]:
ourOutput = ourMSE.forward(x, torch.zeros_like(x))
torchOutput = torchMSE(x, torch.zeros_like(x))
torch.testing.assert_allclose(torchOutput, ourOutput)

In [63]:
#test not written correctly
ourBackward = ourMSE.backward()
torchBackward = torchOutput.backward()
torch.testing.assert_allclose(x.grad, ourBackward)

# ReLu test

In [97]:
ourReLU = ReLU()
torchReLU = nn.ReLU()
x=Variable(randomStuff,requires_grad=True)

In [98]:
ourOutput = ourReLU.forward(x)
torchOutput = torchReLU(x)
torch.testing.assert_allclose(torchOutput, ourOutput)

In [99]:
grdout=torch.rand_like(x)

In [104]:
grdout.shape

torch.Size([10, 3, 25, 25])

In [100]:
torchOutput.backward(grdout)

In [101]:
torch.testing.assert_allclose(ourReLU.backward(grdout),x.grad)

In [102]:
ourReLU.backward(grdout).shape

torch.Size([10, 3, 25, 25])

# sigmoid test

In [107]:
ourSigmoid = Sigmoid()
torchSigmoid = nn.Sigmoid()
x=Variable(randomStuff,requires_grad=True)
grdout=torch.rand_like(x)

In [108]:
ourOutput = ourSigmoid.forward(x)
torchOutput = torchSigmoid(x)
torch.testing.assert_allclose(torchOutput, ourOutput)

In [109]:
grdout.shape

torch.Size([10, 3, 25, 25])

In [110]:
torchOutput.backward(grdout)

In [111]:
torch.testing.assert_allclose(ourSigmoid.backward(grdout),x.grad)

In [112]:
ourSigmoid.backward(grdout).shape

torch.Size([10, 3, 25, 25])

# Sequential test

In [8]:
ourSequential = Sequential(ourConv, ReLU(), Sigmoid())
torchSequential = nn.Sequential( torchConv, nn.ReLU(), nn.Sigmoid())
x=Variable(randomStuff,requires_grad=True)

In [28]:
ourSequential = Sequential(ourConv)
torchSequential = nn.Sequential( torchConv)
x=Variable(randomStuff,requires_grad=True)

In [9]:
ourOutput = ourSequential.forward(x)
torchOutput = torchSequential(x)
torch.testing.assert_allclose(torchOutput, ourOutput)

In [10]:
grdout=torch.rand_like(ourOutput)

In [11]:
torchOutput.backward(grdout)

In [12]:
grdout.shape

torch.Size([10, 48, 9, 9])

In [13]:
ourSequential.backward(grdout)

torch.Size([48, 9, 9])
torch.Size([48, 9, 9])
torch.Size([48, 9, 9])


In [14]:
stuff=ourConv.dweights

In [17]:
torch.testing.assert_allclose(torchConv.weight.grad,stuff)

In [18]:
torch.testing.assert_allclose(torchConv.bias.grad,ourConv.dbias)

# Full model

In [9]:
noisy_imgs_1, noisy_imgs_2 = torch.load('data/train_data.pkl') #pairs of images with different noises each time
val_noisy_imgs, clean_imgs = torch.load('data/val_data.pkl')
val_noisy_imgs = val_noisy_imgs
clean_imgs = clean_imgs
train_input, train_target = ((noisy_imgs_1/255.0)), ((noisy_imgs_2/255.0))


In [10]:
val_imgs_during_training = val_noisy_imgs[800:]
clean_imgs_during_training = clean_imgs[800:]

In [11]:
def psnr ( denoised , ground_truth ) :
    # Peak Signal to Noise Ratio : denoised and ground_truth have range [0 , 1]
    mse = torch . mean (( denoised - ground_truth ) ** 2)
    return -10 * torch . log10 ( mse + 10** -8)

def validation_check(model):
    with torch.no_grad():
        denoised = ((model(val_imgs_during_training/255)))
        avg_psnr = 0
        for i in range(200) :
             avg_psnr += psnr(denoised[i], clean_imgs_during_training[i]/255)
        return avg_psnr/200

In [32]:
kernel = 3
stride = 2
        
model = Sequential(
    Conv2d(3,48,kernel,stride),
    ReLU(),
    Conv2d(48,48,kernel,stride),
    ReLU(),
    Conv2d(48,48,kernel,stride = 1, padding = 2),
    NNUpsampling(scale_factor = 2),
    Conv2d(48,24,kernel,stride = 1,),
    ReLU(),
    Conv2d(24,24,kernel,stride = 1, padding = 2),
    NNUpsampling(scale_factor = 2),
    Conv2d(24,24,kernel,stride = 1,),
    Conv2d(24,3,kernel,stride = 1,),
    Sigmoid()
)

In [33]:
optimizer = SGD(model.params(), lr = 0.15)

In [34]:
loss = MSE()

In [ ]:
epochs = 17
batch_size = 32

for i in range(epochs):
    for b in range(0, len(train_input), batch_size):
        optimizer.zero_grad()
        data = train_input[b:b+batch_size]
        target = train_target[b:b+batch_size]
        output=loss(model(data),target)
        model.backward(loss.backward())
        optimizer.step()
        print(b/32)
    val = validation_check(model)
    print(val)

0.0
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
17.0
18.0
19.0
20.0
21.0
22.0
23.0
24.0
25.0
26.0
27.0
28.0
29.0
30.0
31.0
32.0
33.0
34.0
35.0
36.0
37.0
38.0
39.0
40.0
41.0
42.0
43.0
44.0
45.0
46.0
47.0
48.0
49.0
50.0
51.0
52.0
53.0
54.0
55.0
56.0
57.0
58.0
59.0
60.0
61.0
62.0
63.0
64.0
65.0
66.0
67.0
68.0
69.0
70.0
71.0
72.0
73.0
74.0
75.0
76.0
77.0
78.0
79.0
80.0
81.0
82.0
83.0
84.0
85.0
86.0
87.0
88.0
89.0
90.0
91.0
92.0
93.0
94.0
95.0
96.0
97.0
98.0
99.0
100.0
101.0
102.0
103.0
104.0
105.0
106.0
107.0
108.0
109.0
110.0
111.0
112.0
113.0
114.0
115.0
116.0
117.0
118.0
119.0
120.0
121.0
122.0
123.0
124.0
125.0
126.0
127.0
128.0
129.0
130.0
131.0
132.0
133.0
134.0
135.0
136.0
137.0
138.0
139.0
140.0
141.0
142.0
143.0
144.0
145.0
146.0
147.0
148.0
149.0
150.0
151.0
152.0
153.0
154.0
155.0
156.0
157.0
158.0
159.0
160.0
161.0
162.0
163.0
164.0
165.0
166.0
167.0
168.0
169.0
170.0
171.0
172.0
173.0
174.0
175.0
176.0
177.0
178.0
179.0
180.0
181.0
182.0
183.0
184.0
